## Exercise:

Write pandas code that reports the most experienced employees in each project. In case of a tie, report all employees with the maximum number of experience years.

Return the result table in any order.

The query result format is in the following example.

In [5]:
import pandas as pd
import numpy as np
# Users table
users = pd.DataFrame({
    'user_id': [1, 2, 3, 4],
    'join_date': ['2018-01-01', '2018-02-09', '2018-01-19', '2018-05-21'],
    'favorite_brand': ['Lenovo', 'Samsung', 'LG', 'HP']
})

# Orders table
orders = pd.DataFrame({
    'order_id': [1, 2, 3, 4, 5, 6],
    'order_date': ['2019-08-01', '2018-08-02', '2019-08-03', '2018-08-04', '2018-08-04', '2019-08-05'],
    'item_id': [4, 2, 3, 1, 1, 2],
    'buyer_id': [1, 1, 2, 4, 3, 2],
    'seller_id': [2, 3, 3, 2, 4, 4]
})

# Items table
items = pd.DataFrame({
    'item_id': [1, 2, 3, 4],
    'item_brand': ['Samsung', 'Lenovo', 'LG', 'HP']
})

In [6]:
Project.describe()

,project_id,employee_id
count,5.000000,5.00000
mean,1.400000,2.20000
std,0.547723,1.30384
min,1.000000,1.00000
25%,1.000000,1.00000
50%,1.000000,2.00000
75%,2.000000,3.00000
max,2.000000,4.00000


In [7]:
Project.dtypes

project_id     int64
employee_id    int64
dtype: object

In [8]:
Project.columns

Index(['project_id', 'employee_id'], dtype='object')

In [9]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [10]:
cond = orders['order_date'].dt.year == 2019
orders_only_2019 = orders[cond]
orders_only_2019

,order_id,order_date,item_id,buyer_id,seller_id
0,1,2019-08-01,4,1,2
2,3,2019-08-03,3,2,3
5,6,2019-08-05,2,2,4


In [11]:
inner_joined = users.merge(orders_only_2019, left_on='user_id', right_on='buyer_id', how='left')

In [12]:
inner_joined

,user_id,join_date,favorite_brand,order_id,order_date,item_id,buyer_id,seller_id
0,1,2018-01-01,Lenovo,1.0,2019-08-01,4.0,1.0,2.0
1,2,2018-02-09,Samsung,3.0,2019-08-03,3.0,2.0,3.0
2,2,2018-02-09,Samsung,6.0,2019-08-05,2.0,2.0,4.0
3,3,2018-01-19,LG,NaN,NaT,NaN,NaN,NaN
4,4,2018-05-21,HP,NaN,NaT,NaN,NaN,NaN


In [13]:
inner_joined['order_to_aggregaete'] = np.where(np.isnan(inner_joined['buyer_id']), 0, 1)
inner_joined

,user_id,join_date,favorite_brand,order_id,order_date,item_id,buyer_id,seller_id,order_to_aggregaete
0,1,2018-01-01,Lenovo,1.0,2019-08-01,4.0,1.0,2.0,1
1,2,2018-02-09,Samsung,3.0,2019-08-03,3.0,2.0,3.0,1
2,2,2018-02-09,Samsung,6.0,2019-08-05,2.0,2.0,4.0,1
3,3,2018-01-19,LG,NaN,NaT,NaN,NaN,NaN,0
4,4,2018-05-21,HP,NaN,NaT,NaN,NaN,NaN,0


In [14]:
result_df =inner_joined.groupby(['user_id', 'join_date']).agg(orders_in_2019=('order_to_aggregaete', 'sum')).reset_index().rename(columns={
    'user_id': 'buyer_id'
})
result_df

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01,1
1,2,2018-02-09,2
2,3,2018-01-19,0
3,4,2018-05-21,0


In [15]:
# SECOND SOLUTION
inner_joined = users.merge(orders_only_2019, left_on='user_id', right_on='buyer_id', how='left')
inner_joined

,user_id,join_date,favorite_brand,order_id,order_date,item_id,buyer_id,seller_id
0,1,2018-01-01,Lenovo,1.0,2019-08-01,4.0,1.0,2.0
1,2,2018-02-09,Samsung,3.0,2019-08-03,3.0,2.0,3.0
2,2,2018-02-09,Samsung,6.0,2019-08-05,2.0,2.0,4.0
3,3,2018-01-19,LG,NaN,NaT,NaN,NaN,NaN
4,4,2018-05-21,HP,NaN,NaT,NaN,NaN,NaN


In [16]:
inner_joined["null_flag"] = ~inner_joined["buyer_id"].isna()
inner_joined

,user_id,join_date,favorite_brand,order_id,order_date,item_id,buyer_id,seller_id,null_flag
0,1,2018-01-01,Lenovo,1.0,2019-08-01,4.0,1.0,2.0,True
1,2,2018-02-09,Samsung,3.0,2019-08-03,3.0,2.0,3.0,True
2,2,2018-02-09,Samsung,6.0,2019-08-05,2.0,2.0,4.0,True
3,3,2018-01-19,LG,NaN,NaT,NaN,NaN,NaN,False
4,4,2018-05-21,HP,NaN,NaT,NaN,NaN,NaN,False


In [17]:
second_sol = inner_joined.groupby(["user_id", 'join_date']).agg(orders_in_2019=('null_flag', 'sum')).reset_index().rename(columns={
    "user_id": "buyer_id"
})
second_sol

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01,1
1,2,2018-02-09,2
2,3,2018-01-19,0
3,4,2018-05-21,0


In [18]:
merged_df = pd.merge(users, orders_only_2019[['buyer_id', 'order_date']], how='left', left_on='user_id', right_on='buyer_id')

merged_df

,user_id,join_date,favorite_brand,buyer_id,order_date
0,1,2018-01-01,Lenovo,1.0,2019-08-01
1,2,2018-02-09,Samsung,2.0,2019-08-03
2,2,2018-02-09,Samsung,2.0,2019-08-05
3,3,2018-01-19,LG,NaN,NaT
4,4,2018-05-21,HP,NaN,NaT


In [19]:
# group by user_id and join_date and calculate count of orders made in 2019
result_df = merged_df.groupby(['user_id', 'join_date'])[['buyer_id']].count().reset_index()
result_df

,user_id,join_date,buyer_id
0,1,2018-01-01,1
1,2,2018-02-09,2
2,3,2018-01-19,0
3,4,2018-05-21,0


In [20]:
result_df.rename(columns={'order_date': 'orders_in_2019', 'user_id': 'buyer_id'}, inplace=True)
result_df.fillna(0, inplace=True) # fill null values with 0
result_df

,buyer_id,join_date,buyer_id
0,1,2018-01-01,1
1,2,2018-02-09,2
2,3,2018-01-19,0
3,4,2018-05-21,0


In [46]:
result_df['orders_in_2019'] =  result_df['orders_in_2019'].astype('int64')

In [47]:
expected_df = pd.DataFrame({
        'buyer_id': [1, 2, 3, 4],
        'join_date': ['2018-01-01', '2018-02-09', '2018-01-19', '2018-05-21'],
        'orders_in_2019': [1, 2, 0, 0]
    })
expected_df

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01,1
1,2,2018-02-09,2
2,3,2018-01-19,0
3,4,2018-05-21,0


In [64]:
pd.testing.assert_frame_equal(second_sol, expected_df)